In [151]:
from torch.utils.data import DataLoader
# text embedding - >
import pandas as pd
from sklearn.preprocessing import LabelEncoder
# Word2vec , FastText 등 embedding 방법론 적용 or random embedd 하는 keras embedding 사용
from tensorflow.keras.layers import Embedding
from transformers import BertConfig, BertModel
import sentencepiece as spm
import csv
import re

In [152]:
from transformers import BertTokenizer

In [153]:
# 전처리된 상품명을 인덱스로 변환하여 token_ids를 만들었음  token_dict
# 전처리된 상품명을 하나의 텍스트벡터(text_vec)로 변환 BertModel
# 반환 튜플(시퀀스 아웃풋, 풀드(pooled) 아웃풋) 중 시퀀스 아웃풋만 사용

# # bertmodel 로 embedding vector


In [156]:

train = pd.read_csv("./dataset/train.csv")

In [157]:
le = LabelEncoder()

In [158]:
train['discourse_type'] = le.fit_transform(train['discourse_type'].values.tolist())

In [159]:
train['discourse_text'].values.tolist()

["Hi, i'm Isaac, i'm going to be writing about how this face on Mars is a natural landform or if there is life on Mars that made it. The story is about how NASA took a picture of Mars and a face was seen on the planet. NASA doesn't know if the landform was created by life on Mars, or if it is just a natural landform. ",
 "On my perspective, I think that the face is a natural landform because I dont think that there is any life on Mars. In these next few paragraphs, I'll be talking about how I think that is is a natural landform ",
 'I think that the face is a natural landform because there is no life on Mars that we have descovered yet ',
 'If life was on Mars, we would know by now. The reason why I think it is a natural landform because, nobody live on Mars in order to create the figure. It says in paragraph 9, "It\'s not easy to target Cydonia," in which he is saying that its not easy to know if it is a natural landform at this point. In all that they\'re saying, its probably a natur

In [160]:
train

,discourse_id,essay_id,discourse_text,discourse_type,discourse_effectiveness
0,0013cc385424,007ACE74B050,"Hi, i'm Isaac, i'm going to be writing about h...",4,Adequate
1,9704a709b505,007ACE74B050,"On my perspective, I think that the face is a ...",5,Adequate
2,c22adee811b6,007ACE74B050,I think that the face is a natural landform be...,0,Adequate
3,a10d361e54e4,007ACE74B050,"If life was on Mars, we would know by now. The...",3,Adequate
4,db3e453ec4e2,007ACE74B050,People thought that the face was formed by ali...,2,Adequate
...,...,...,...,...,...
36760,9f63b687e76a,FFA381E58FC6,For many people they don't like only asking on...,0,Adequate
36761,9d5bd7d86212,FFA381E58FC6,also people have different views and opinions ...,0,Adequate
36762,f1b78becd573,FFA381E58FC6,Advice is something that can impact a persons ...,5,Adequate
36763,cc184624ca8e,FFA381E58FC6,someone can use everything that many people sa...,3,Ineffective


In [163]:
#총 5만개의 샘플이 존재합니다. 센텐스피스의 입력으로 사용하기 위해서 데이터프레임을 txt 파일로 저장합니다.
with open('train.txt', 'w', encoding='utf-8') as f:
    f.write("\n".join(train['discourse_text']))

In [164]:
spm.SentencePieceTrainer.Train('--input=train.txt --model_prefix=feedback --vocab_size=5000 --model_type=bpe --max_sentence_length=9999')

In [165]:
vocab_list = pd.read_csv('feedback.vocab', sep='\t', header=None, quoting=csv.QUOTE_NONE)
vocab_list.sample(10)

,0,1
2931,umbia,-2928
670,▁dang,-667
2575,▁communicate,-2572
1586,ified,-1583
4410,▁absolutely,-4407
2782,▁18,-2779
1644,rict,-1641
1334,river,-1331
2612,▁surpr,-2609
3329,atory,-3326


In [166]:
sp = spm.SentencePieceProcessor()
vocab_file = "feedback.model"
sp.load(vocab_file)

True

True

In [167]:
import os
VOCAB_DIR = ''
sp.Load('feedback.model')

True

In [168]:


for text in train['discourse_text'].values.tolist():
    print(text)
    print(sp.encode_as_pieces(text))
    print(sp.encode_as_ids(text))

    break

Hi, i'm Isaac, i'm going to be writing about how this face on Mars is a natural landform or if there is life on Mars that made it. The story is about how NASA took a picture of Mars and a face was seen on the planet. NASA doesn't know if the landform was created by life on Mars, or if it is just a natural landform. 
['▁H', 'i', ',', '▁i', "'", 'm', '▁Is', 'a', 'ac', ',', '▁i', "'", 'm', '▁going', '▁to', '▁be', '▁writing', '▁about', '▁how', '▁this', '▁face', '▁on', '▁Mars', '▁is', '▁a', '▁natural', '▁landform', '▁or', '▁if', '▁there', '▁is', '▁life', '▁on', '▁Mars', '▁that', '▁made', '▁it', '.', '▁The', '▁story', '▁is', '▁about', '▁how', '▁NASA', '▁took', '▁a', '▁picture', '▁of', '▁Mars', '▁and', '▁a', '▁face', '▁was', '▁seen', '▁on', '▁the', '▁planet', '.', '▁NASA', '▁doesn', "'", 't', '▁know', '▁if', '▁the', '▁landform', '▁was', '▁created', '▁by', '▁life', '▁on', '▁Mars', ',', '▁or', '▁if', '▁it', '▁is', '▁just', '▁a', '▁natural', '▁landform', '.']
[363, 4939, 4956, 19, 4959, 4946, 18

In [169]:
sp.encode_as_pieces(text)

['▁H',
 'i',
 ',',
 '▁i',
 "'",
 'm',
 '▁Is',
 'a',
 'ac',
 ',',
 '▁i',
 "'",
 'm',
 '▁going',
 '▁to',
 '▁be',
 '▁writing',
 '▁about',
 '▁how',
 '▁this',
 '▁face',
 '▁on',
 '▁Mars',
 '▁is',
 '▁a',
 '▁natural',
 '▁landform',
 '▁or',
 '▁if',
 '▁there',
 '▁is',
 '▁life',
 '▁on',
 '▁Mars',
 '▁that',
 '▁made',
 '▁it',
 '.',
 '▁The',
 '▁story',
 '▁is',
 '▁about',
 '▁how',
 '▁NASA',
 '▁took',
 '▁a',
 '▁picture',
 '▁of',
 '▁Mars',
 '▁and',
 '▁a',
 '▁face',
 '▁was',
 '▁seen',
 '▁on',
 '▁the',
 '▁planet',
 '.',
 '▁NASA',
 '▁doesn',
 "'",
 't',
 '▁know',
 '▁if',
 '▁the',
 '▁landform',
 '▁was',
 '▁created',
 '▁by',
 '▁life',
 '▁on',
 '▁Mars',
 ',',
 '▁or',
 '▁if',
 '▁it',
 '▁is',
 '▁just',
 '▁a',
 '▁natural',
 '▁landform',
 '.']

In [125]:
train['discourse_text']

0        Hi, i'm Isaac, i'm going to be writing about h...
1        On my perspective, I think that the face is a ...
2        I think that the face is a natural landform be...
3        If life was on Mars, we would know by now. The...
4        People thought that the face was formed by ali...
                               ...                        
36760    For many people they don't like only asking on...
36761    also people have different views and opinions ...
36762    Advice is something that can impact a persons ...
36763    someone can use everything that many people sa...
36764    In conclusion asking for an opinion can be ben...
Name: discourse_text, Length: 36765, dtype: object

In [170]:
#train['tokens'] = train['discourse_text'].map(lambda x: sp.encode_as_pieces(x))
train['tokens'] = train['discourse_text'].map(lambda  x: sp.encode_as_pieces(x))

In [196]:
tokens = train['tokens'].iloc[0]

In [197]:
p = re.compile('▁[^▁]+')  #

In [198]:
tokens = ''.join(tokens)

In [199]:
tokens= p.findall(tokens)

In [200]:
tokens

['▁Hi,',
 "▁i'm",
 '▁Isaac,',
 "▁i'm",
 '▁going',
 '▁to',
 '▁be',
 '▁writing',
 '▁about',
 '▁how',
 '▁this',
 '▁face',
 '▁on',
 '▁Mars',
 '▁is',
 '▁a',
 '▁natural',
 '▁landform',
 '▁or',
 '▁if',
 '▁there',
 '▁is',
 '▁life',
 '▁on',
 '▁Mars',
 '▁that',
 '▁made',
 '▁it.',
 '▁The',
 '▁story',
 '▁is',
 '▁about',
 '▁how',
 '▁NASA',
 '▁took',
 '▁a',
 '▁picture',
 '▁of',
 '▁Mars',
 '▁and',
 '▁a',
 '▁face',
 '▁was',
 '▁seen',
 '▁on',
 '▁the',
 '▁planet.',
 '▁NASA',
 "▁doesn't",
 '▁know',
 '▁if',
 '▁the',
 '▁landform',
 '▁was',
 '▁created',
 '▁by',
 '▁life',
 '▁on',
 '▁Mars,',
 '▁or',
 '▁if',
 '▁it',
 '▁is',
 '▁just',
 '▁a',
 '▁natural',
 '▁landform.']

In [201]:
token_types = [type_id for type_id, word in enumerate(tokens) for _ in word.split()]

['▁Hi,',
 "▁i'm",
 '▁Isaac,',
 "▁i'm",
 '▁going',
 '▁to',
 '▁be',
 '▁writing',
 '▁about',
 '▁how',
 '▁this',
 '▁face',
 '▁on',
 '▁Mars',
 '▁is',
 '▁a',
 '▁natural',
 '▁landform',
 '▁or',
 '▁if',
 '▁there',
 '▁is',
 '▁life',
 '▁on',
 '▁Mars',
 '▁that',
 '▁made',
 '▁it.',
 '▁The',
 '▁story',
 '▁is',
 '▁about',
 '▁how',
 '▁NASA',
 '▁took',
 '▁a',
 '▁picture',
 '▁of',
 '▁Mars',
 '▁and',
 '▁a',
 '▁face',
 '▁was',
 '▁seen',
 '▁on',
 '▁the',
 '▁planet.',
 '▁NASA',
 "▁doesn't",
 '▁know',
 '▁if',
 '▁the',
 '▁landform',
 '▁was',
 '▁created',
 '▁by',
 '▁life',
 '▁on',
 '▁Mars,',
 '▁or',
 '▁if',
 '▁it',
 '▁is',
 '▁just',
 '▁a',
 '▁natural',
 '▁landform.']

In [202]:
token_types

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66]

In [186]:


for text in train['discourse_text'].values.tolist():
    print(text)
    print(sp.encode_as_pieces(text))
    print(sp.encode_as_ids(text))

    break

Hi, i'm Isaac, i'm going to be writing about how this face on Mars is a natural landform or if there is life on Mars that made it. The story is about how NASA took a picture of Mars and a face was seen on the planet. NASA doesn't know if the landform was created by life on Mars, or if it is just a natural landform. 
['▁H', 'i', ',', '▁i', "'", 'm', '▁Is', 'a', 'ac', ',', '▁i', "'", 'm', '▁going', '▁to', '▁be', '▁writing', '▁about', '▁how', '▁this', '▁face', '▁on', '▁Mars', '▁is', '▁a', '▁natural', '▁landform', '▁or', '▁if', '▁there', '▁is', '▁life', '▁on', '▁Mars', '▁that', '▁made', '▁it', '.', '▁The', '▁story', '▁is', '▁about', '▁how', '▁NASA', '▁took', '▁a', '▁picture', '▁of', '▁Mars', '▁and', '▁a', '▁face', '▁was', '▁seen', '▁on', '▁the', '▁planet', '.', '▁NASA', '▁doesn', "'", 't', '▁know', '▁if', '▁the', '▁landform', '▁was', '▁created', '▁by', '▁life', '▁on', '▁Mars', ',', '▁or', '▁if', '▁it', '▁is', '▁just', '▁a', '▁natural', '▁landform', '.']
[363, 4939, 4956, 19, 4959, 4946, 18

In [114]:
import os

In [115]:
class CFG:
    batch_size=1024 # 배치 사이즈
    num_workers=4 # 워커의 개수
    print_freq=100 # 결과 출력 빈도
    warmup_steps=100 # lr을 서서히 증가시킬 step 수
    hidden_size=512 # 은닉 크기
    dropout=0.2 # dropout 확률
    intermediate_size=256 # TRANSFORMER셀의 intermediate 크기
    nlayers=2 # BERT의 층수
    nheads=8 # BERT의 head 개수
    seq_len=64 # 토큰의 최대 길이
    n_b_cls = 57 + 1 # 대카테고리 개수
    n_m_cls = 552 + 1 # 중카테고리 개수
    n_s_cls = 3190 + 1 # 소카테고리 개수
    n_d_cls = 404 + 1 # 세카테고리 개수
    vocab_size = 32000 # 토큰의 유니크 인덱스 개수
    img_feat_size = 2048 # 이미지 피처 벡터의 크기
    type_vocab_size = 30 # 타입의 유니크 인덱스 개수
    #csv_path = os.path.join(DB_DIR, 'dev.csv') # 전처리 돼 저장된 dev 데이터셋
    #h5_path = os.path.join(DB_DIR, 'dev_img_feat.h5')

In [116]:
cfg = CFG()

bert_cfg= BertConfig(
        cfg.vocab_size, # 사전 크기
        hidden_size=cfg.hidden_size, # 히든 크기
        num_hidden_layers=cfg.nlayers, # 레이어 층 수
        num_attention_heads=cfg.nheads, # 어텐션 헤드의 수
        intermediate_size=cfg.intermediate_size, # 인터미디어트 크기
        hidden_dropout_prob=cfg.dropout, # 히든 드롭아웃 확률 값
        attention_probs_dropout_prob=cfg.dropout, # 어텐션 드롭아웃 확률 값
        max_position_embeddings=cfg.seq_len, # 포지션 임베딩의 최대 길이
        type_vocab_size=cfg.type_vocab_size, # 타입 사전 크기
)

In [195]:
text_encoder= BertModel(config= bert_cfg)

In [205]:
train['input_ids'] = train['discourse_text'].map(lambda x: sp.encode_as_ids(x))

In [119]:
train

,discourse_id,essay_id,discourse_text,discourse_type,discourse_effectiveness,tokens,input_ids
0,0013cc385424,007ACE74B050,"Hi, i'm Isaac, i'm going to be writing about h...",4,Adequate,"[▁H, i, ,, ▁i, ', m, ▁Is, a, ac, ,, ▁i, ', m, ...","[363, 4939, 4956, 19, 4959, 4946, 1899, 4936, ..."
1,9704a709b505,007ACE74B050,"On my perspective, I think that the face is a ...",5,Adequate,"[▁On, ▁my, ▁perspective, ,, ▁I, ▁think, ▁that,...","[642, 353, 2764, 4956, 70, 228, 51, 7, 433, 53..."
2,c22adee811b6,007ACE74B050,I think that the face is a natural landform be...,0,Adequate,"[▁I, ▁think, ▁that, ▁the, ▁face, ▁is, ▁a, ▁nat...","[70, 228, 51, 7, 433, 53, 5, 1066, 849, 146, 2..."
3,a10d361e54e4,007ACE74B050,"If life was on Mars, we would know by now. The...",3,Adequate,"[▁If, ▁life, ▁was, ▁on, ▁Mars, ,, ▁we, ▁would,...","[357, 441, 193, 66, 652, 4956, 103, 110, 243, ..."
4,db3e453ec4e2,007ACE74B050,People thought that the face was formed by ali...,2,Adequate,"[▁People, ▁thought, ▁that, ▁the, ▁face, ▁was, ...","[1048, 933, 51, 7, 433, 193, 2855, 212, 138, 3..."
...,...,...,...,...,...,...,...
36760,9f63b687e76a,FFA381E58FC6,For many people they don't like only asking on...,0,Adequate,"[▁For, ▁many, ▁people, ▁they, ▁don, ', t, ▁lik...","[600, 324, 125, 77, 292, 4959, 4934, 190, 378,..."
36761,9d5bd7d86212,FFA381E58FC6,also people have different views and opinions ...,0,Adequate,"[▁also, ▁people, ▁have, ▁different, ▁views, ▁a...","[293, 125, 89, 487, 3027, 36, 831, 66, 408, 4954]"
36762,f1b78becd573,FFA381E58FC6,Advice is something that can impact a persons ...,5,Adequate,"[▁Ad, vice, ▁is, ▁something, ▁that, ▁can, ▁imp...","[1971, 335, 53, 416, 51, 91, 1893, 5, 3000, 12..."
36763,cc184624ca8e,FFA381E58FC6,someone can use everything that many people sa...,3,Ineffective,"[▁someone, ▁can, ▁use, ▁everything, ▁that, ▁ma...","[557, 91, 403, 1206, 51, 324, 125, 710, 36, 40..."


In [24]:
tokens_max_len = max([len(text) for text in train['discourse_text']])

In [25]:
train_mask = train['discourse_text'].map(lambda x :[1] * len(x) )

KeyError: 'input_ids'

In [ ]:
token_ids = train['input_ids'].iloc[0]

In [204]:
train

,discourse_id,essay_id,discourse_text,discourse_type,discourse_effectiveness,tokens
0,0013cc385424,007ACE74B050,"Hi, i'm Isaac, i'm going to be writing about h...",4,Adequate,"[▁H, i, ,, ▁i, ', m, ▁Is, a, ac, ,, ▁i, ', m, ..."
1,9704a709b505,007ACE74B050,"On my perspective, I think that the face is a ...",5,Adequate,"[▁On, ▁my, ▁perspective, ,, ▁I, ▁think, ▁that,..."
2,c22adee811b6,007ACE74B050,I think that the face is a natural landform be...,0,Adequate,"[▁I, ▁think, ▁that, ▁the, ▁face, ▁is, ▁a, ▁nat..."
3,a10d361e54e4,007ACE74B050,"If life was on Mars, we would know by now. The...",3,Adequate,"[▁If, ▁life, ▁was, ▁on, ▁Mars, ,, ▁we, ▁would,..."
4,db3e453ec4e2,007ACE74B050,People thought that the face was formed by ali...,2,Adequate,"[▁People, ▁thought, ▁that, ▁the, ▁face, ▁was, ..."
...,...,...,...,...,...,...
36760,9f63b687e76a,FFA381E58FC6,For many people they don't like only asking on...,0,Adequate,"[▁For, ▁many, ▁people, ▁they, ▁don, ', t, ▁lik..."
36761,9d5bd7d86212,FFA381E58FC6,also people have different views and opinions ...,0,Adequate,"[▁also, ▁people, ▁have, ▁different, ▁views, ▁a..."
36762,f1b78becd573,FFA381E58FC6,Advice is something that can impact a persons ...,5,Adequate,"[▁Ad, vice, ▁is, ▁something, ▁that, ▁can, ▁imp..."
36763,cc184624ca8e,FFA381E58FC6,someone can use everything that many people sa...,3,Ineffective,"[▁someone, ▁can, ▁use, ▁everything, ▁that, ▁ma..."


In [206]:

token_ids

[363,
 4939,
 4956,
 19,
 4959,
 4946,
 1899,
 4936,
 1300,
 4956,
 19,
 4959,
 4946,
 446,
 23,
 32,
 2290,
 239,
 268,
 154,
 433,
 66,
 652,
 53,
 5,
 1066,
 849,
 116,
 160,
 208,
 53,
 441,
 66,
 652,
 51,
 612,
 62,
 4954,
 114,
 1673,
 53,
 239,
 268,
 799,
 1513,
 5,
 1080,
 38,
 652,
 36,
 5,
 433,
 193,
 1639,
 66,
 7,
 459,
 4954,
 799,
 1096,
 4959,
 4934,
 243,
 160,
 7,
 849,
 193,
 1208,
 212,
 441,
 66,
 652,
 4956,
 116,
 160,
 62,
 53,
 236,
 5,
 1066,
 849,
 4954,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


In [207]:
token_mask = [1] * len(token_ids)

In [34]:
token_mask


[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1]

In [208]:
token_pad = [0] * (tokens_max_len - len(token_ids))
token_ids += token_pad
token_mask += token_pad

,discourse_id,essay_id,discourse_text,discourse_type,discourse_effectiveness,input_ids
0,0013cc385424,007ACE74B050,"Hi, i'm Isaac, i'm going to be writing about h...",4,Adequate,"[363, 4939, 4956, 19, 4959, 4946, 1899, 4936, ..."
1,9704a709b505,007ACE74B050,"On my perspective, I think that the face is a ...",5,Adequate,"[642, 353, 2764, 4956, 70, 228, 51, 7, 433, 53..."
2,c22adee811b6,007ACE74B050,I think that the face is a natural landform be...,0,Adequate,"[70, 228, 51, 7, 433, 53, 5, 1066, 849, 146, 2..."
3,a10d361e54e4,007ACE74B050,"If life was on Mars, we would know by now. The...",3,Adequate,"[357, 441, 193, 66, 652, 4956, 103, 110, 243, ..."
4,db3e453ec4e2,007ACE74B050,People thought that the face was formed by ali...,2,Adequate,"[1048, 933, 51, 7, 433, 193, 2855, 212, 138, 3..."
...,...,...,...,...,...,...
36760,9f63b687e76a,FFA381E58FC6,For many people they don't like only asking on...,0,Adequate,"[600, 324, 125, 77, 292, 4959, 4934, 190, 378,..."
36761,9d5bd7d86212,FFA381E58FC6,also people have different views and opinions ...,0,Adequate,"[293, 125, 89, 487, 3027, 36, 831, 66, 408, 4954]"
36762,f1b78becd573,FFA381E58FC6,Advice is something that can impact a persons ...,5,Adequate,"[1971, 335, 53, 416, 51, 91, 1893, 5, 3000, 12..."
36763,cc184624ca8e,FFA381E58FC6,someone can use everything that many people sa...,3,Ineffective,"[557, 91, 403, 1206, 51, 324, 125, 710, 36, 40..."


In [ ]:
#seg_tokens = train['tokens'][0]

In [130]:
#seg_tokens = ' '.join(train['tokens'].iloc[0])

In [131]:
token_types = [type_id for type_id, word in enumerate(seg_tokens) for _ in word.split()]

In [38]:
#train['tokens'] = train['discourse_text'].map(lambda x: sp.encode_as_pieces(x))
train['tokens'] = train['discourse_text'].map(sp.EncodeAsPieces)

,discourse_id,essay_id,discourse_text,discourse_type,discourse_effectiveness,input_ids
0,0013cc385424,007ACE74B050,"Hi, i'm Isaac, i'm going to be writing about h...",4,Adequate,"[363, 4939, 4956, 19, 4959, 4946, 1899, 4936, ..."
1,9704a709b505,007ACE74B050,"On my perspective, I think that the face is a ...",5,Adequate,"[642, 353, 2764, 4956, 70, 228, 51, 7, 433, 53..."
2,c22adee811b6,007ACE74B050,I think that the face is a natural landform be...,0,Adequate,"[70, 228, 51, 7, 433, 53, 5, 1066, 849, 146, 2..."
3,a10d361e54e4,007ACE74B050,"If life was on Mars, we would know by now. The...",3,Adequate,"[357, 441, 193, 66, 652, 4956, 103, 110, 243, ..."
4,db3e453ec4e2,007ACE74B050,People thought that the face was formed by ali...,2,Adequate,"[1048, 933, 51, 7, 433, 193, 2855, 212, 138, 3..."
...,...,...,...,...,...,...
36760,9f63b687e76a,FFA381E58FC6,For many people they don't like only asking on...,0,Adequate,"[600, 324, 125, 77, 292, 4959, 4934, 190, 378,..."
36761,9d5bd7d86212,FFA381E58FC6,also people have different views and opinions ...,0,Adequate,"[293, 125, 89, 487, 3027, 36, 831, 66, 408, 4954]"
36762,f1b78becd573,FFA381E58FC6,Advice is something that can impact a persons ...,5,Adequate,"[1971, 335, 53, 416, 51, 91, 1893, 5, 3000, 12..."
36763,cc184624ca8e,FFA381E58FC6,someone can use everything that many people sa...,3,Ineffective,"[557, 91, 403, 1206, 51, 324, 125, 710, 36, 40..."


In [ ]:
seg_tokens

In [134]:
token_types

[0,
 1,
 3,
 5,
 7,
 8,
 10,
 12,
 14,
 15,
 16,
 18,
 20,
 21,
 23,
 25,
 26,
 28,
 30,
 32,
 33,
 34,
 35,
 36,
 37,
 39,
 40,
 41,
 43,
 44,
 45,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 56,
 57,
 58,
 59,
 60,
 61,
 63,
 64,
 65,
 66,
 68,
 69,
 70,
 71,
 72,
 74,
 75,
 76,
 77,
 78,
 80,
 81,
 82,
 84,
 85,
 86,
 87,
 88,
 90,
 91,
 92,
 94,
 95,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 116,
 117,
 118,
 120,
 121,
 122,
 124,
 125,
 126,
 127,
 128,
 129,
 131,
 132,
 133,
 135,
 136,
 137,
 138,
 139,
 141,
 142,
 143,
 145,
 146,
 147,
 148,
 149,
 151,
 152,
 153,
 154,
 155,
 157,
 158,
 159,
 160,
 161,
 163,
 164,
 165,
 167,
 169,
 170,
 171,
 172,
 174,
 175,
 176,
 177,
 178,
 179,
 181,
 182,
 183,
 185,
 186,
 187,
 188,
 189,
 190,
 192,
 193,
 194,
 195,
 197,
 198,
 199,
 200,
 201,
 203,
 204,
 205,
 206,
 207,
 209,
 210,
 212,
 213,
 214,
 215,
 216,
 217,
 218,
 219,
 221,
 222,
 223,
 225,
 226,
 227,
 

In [ ]:
train['tokens']
%%
token_types

In [ ]:
vocab = [line.split('\t')[0] for line in open(os.path.join(VOCAB_DIR, 'spm.vocab'), encoding='utf-8').readlines()]
token2id = dict([(w, i) for i, w in enumerate(vocab)])

0        [▁H, i, ,, ▁i, ', m, ▁Is, a, ac, ,, ▁i, ', m, ...
1        [▁On, ▁my, ▁perspective, ,, ▁I, ▁think, ▁that,...
2        [▁I, ▁think, ▁that, ▁the, ▁face, ▁is, ▁a, ▁nat...
3        [▁If, ▁life, ▁was, ▁on, ▁Mars, ,, ▁we, ▁would,...
4        [▁People, ▁thought, ▁that, ▁the, ▁face, ▁was, ...
                               ...                        
36760    [▁For, ▁many, ▁people, ▁they, ▁don, ', t, ▁lik...
36761    [▁also, ▁people, ▁have, ▁different, ▁views, ▁a...
36762    [▁Ad, vice, ▁is, ▁something, ▁that, ▁can, ▁imp...
36763    [▁someone, ▁can, ▁use, ▁everything, ▁that, ▁ma...
36764    [▁In, ▁conclusion, ▁asking, ▁for, ▁an, ▁opinio...
Name: tokens, Length: 36765, dtype: object

In [89]:
train

,discourse_id,essay_id,discourse_text,discourse_type,discourse_effectiveness,input_ids,tokens
0,0013cc385424,007ACE74B050,"Hi, i'm Isaac, i'm going to be writing about h...",4,Adequate,"[363, 4939, 4956, 19, 4959, 4946, 1899, 4936, ...","[▁H, i, ,, ▁i, ', m, ▁Is, a, ac, ,, ▁i, ', m, ..."
1,9704a709b505,007ACE74B050,"On my perspective, I think that the face is a ...",5,Adequate,"[642, 353, 2764, 4956, 70, 228, 51, 7, 433, 53...","[▁On, ▁my, ▁perspective, ,, ▁I, ▁think, ▁that,..."
2,c22adee811b6,007ACE74B050,I think that the face is a natural landform be...,0,Adequate,"[70, 228, 51, 7, 433, 53, 5, 1066, 849, 146, 2...","[▁I, ▁think, ▁that, ▁the, ▁face, ▁is, ▁a, ▁nat..."
3,a10d361e54e4,007ACE74B050,"If life was on Mars, we would know by now. The...",3,Adequate,"[357, 441, 193, 66, 652, 4956, 103, 110, 243, ...","[▁If, ▁life, ▁was, ▁on, ▁Mars, ,, ▁we, ▁would,..."
4,db3e453ec4e2,007ACE74B050,People thought that the face was formed by ali...,2,Adequate,"[1048, 933, 51, 7, 433, 193, 2855, 212, 138, 3...","[▁People, ▁thought, ▁that, ▁the, ▁face, ▁was, ..."
...,...,...,...,...,...,...,...
36760,9f63b687e76a,FFA381E58FC6,For many people they don't like only asking on...,0,Adequate,"[600, 324, 125, 77, 292, 4959, 4934, 190, 378,...","[▁For, ▁many, ▁people, ▁they, ▁don, ', t, ▁lik..."
36761,9d5bd7d86212,FFA381E58FC6,also people have different views and opinions ...,0,Adequate,"[293, 125, 89, 487, 3027, 36, 831, 66, 408, 4954]","[▁also, ▁people, ▁have, ▁different, ▁views, ▁a..."
36762,f1b78becd573,FFA381E58FC6,Advice is something that can impact a persons ...,5,Adequate,"[1971, 335, 53, 416, 51, 91, 1893, 5, 3000, 12...","[▁Ad, vice, ▁is, ▁something, ▁that, ▁can, ▁imp..."
36763,cc184624ca8e,FFA381E58FC6,someone can use everything that many people sa...,3,Ineffective,"[557, 91, 403, 1206, 51, 324, 125, 710, 36, 40...","[▁someone, ▁can, ▁use, ▁everything, ▁that, ▁ma..."


In [68]:
tokens = train['tokens']

In [70]:
tokens[0]

['▁H',
 'i',
 ',',
 '▁i',
 "'",
 'm',
 '▁Is',
 'a',
 'ac',
 ',',
 '▁i',
 "'",
 'm',
 '▁going',
 '▁to',
 '▁be',
 '▁writing',
 '▁about',
 '▁how',
 '▁this',
 '▁face',
 '▁on',
 '▁Mars',
 '▁is',
 '▁a',
 '▁natural',
 '▁landform',
 '▁or',
 '▁if',
 '▁there',
 '▁is',
 '▁life',
 '▁on',
 '▁Mars',
 '▁that',
 '▁made',
 '▁it',
 '.',
 '▁The',
 '▁story',
 '▁is',
 '▁about',
 '▁how',
 '▁NASA',
 '▁took',
 '▁a',
 '▁picture',
 '▁of',
 '▁Mars',
 '▁and',
 '▁a',
 '▁face',
 '▁was',
 '▁seen',
 '▁on',
 '▁the',
 '▁planet',
 '.',
 '▁NASA',
 '▁doesn',
 "'",
 't',
 '▁know',
 '▁if',
 '▁the',
 '▁landform',
 '▁was',
 '▁created',
 '▁by',
 '▁life',
 '▁on',
 '▁Mars',
 ',',
 '▁or',
 '▁if',
 '▁it',
 '▁is',
 '▁just',
 '▁a',
 '▁natural',
 '▁landform',
 '.']

In [96]:
# ^를 [] 안에서쓰면 _ 를 제외하라는 뜻, [] 밖에서 _를 쓰면 _로 시작 하는 패턴을 찾는 것


In [124]:
seg_tokens = ''.join(seg_tokens)

"▁Hi,▁i'm▁Isaac,▁i'm▁going▁to▁be▁writing▁about▁how▁this▁face▁on▁Mars▁is▁a▁natural▁landform▁or▁if▁there▁is▁life▁on▁Mars▁that▁made▁it.▁The▁story▁is▁about▁how▁NASA▁took▁a▁picture▁of▁Mars▁and▁a▁face▁was▁seen▁on▁the▁planet.▁NASA▁doesn't▁know▁if▁the▁landform▁was▁created▁by▁life▁on▁Mars,▁or▁if▁it▁is▁just▁a▁natural▁landform."

In [122]:
# 상품명을 ▁기호를 기준으로 분리하여 파이썬 리스트로 저장
# "▁직소퍼즐 ▁1000 조각 ▁바다 거북 의 ▁여행 ▁pl 12 75" =>
# ["▁직소퍼즐", "▁1000 조각", "▁바다 거북 의", "▁여행", "▁pl 12 75"]
seg_tokens = p.findall(seg_tokens)

TypeError: expected string or bytes-like object

In [ ]:
seg_tokens

In [129]:
tokens

"▁H i , ▁i ' m ▁Is a ac , ▁i ' m ▁going ▁to ▁be ▁writing ▁about ▁how ▁this ▁face ▁on ▁Mars ▁is ▁a ▁natural ▁landform ▁or ▁if ▁there ▁is ▁life ▁on ▁Mars ▁that ▁made ▁it . ▁The ▁story ▁is ▁about ▁how ▁NASA ▁took ▁a ▁picture ▁of ▁Mars ▁and ▁a ▁face ▁was ▁seen ▁on ▁the ▁planet . ▁NASA ▁doesn ' t ▁know ▁if ▁the ▁landform ▁was ▁created ▁by ▁life ▁on ▁Mars , ▁or ▁if ▁it ▁is ▁just ▁a ▁natural ▁landform ."

In [68]:
train['tokens']

0        [▁H, i, ,, ▁i, ', m, ▁Is, a, ac, ,, ▁i, ', m, ...
1        [▁On, ▁my, ▁perspective, ,, ▁I, ▁think, ▁that,...
2        [▁I, ▁think, ▁that, ▁the, ▁face, ▁is, ▁a, ▁nat...
3        [▁If, ▁life, ▁was, ▁on, ▁Mars, ,, ▁we, ▁would,...
4        [▁People, ▁thought, ▁that, ▁the, ▁face, ▁was, ...
                               ...                        
36760    [▁For, ▁many, ▁people, ▁they, ▁don, ', t, ▁lik...
36761    [▁also, ▁people, ▁have, ▁different, ▁views, ▁a...
36762    [▁Ad, vice, ▁is, ▁something, ▁that, ▁can, ▁imp...
36763    [▁someone, ▁can, ▁use, ▁everything, ▁that, ▁ma...
36764    [▁In, ▁conclusion, ▁asking, ▁for, ▁an, ▁opinio...
Name: tokens, Length: 36765, dtype: object

In [63]:
if not isinstance(tokens, str):
    tokens = ''

In [64]:
tokens

"▁H i , ▁i ' m ▁Is a ac , ▁i ' m ▁going ▁to ▁be ▁writing ▁about ▁how ▁this ▁face ▁on ▁Mars ▁is ▁a ▁natural ▁landform ▁or ▁if ▁there ▁is ▁life ▁on ▁Mars ▁that ▁made ▁it . ▁The ▁story ▁is ▁about ▁how ▁NASA ▁took ▁a ▁picture ▁of ▁Mars ▁and ▁a ▁face ▁was ▁seen ▁on ▁the ▁planet . ▁NASA ▁doesn ' t ▁know ▁if ▁the ▁landform ▁was ▁created ▁by ▁life ▁on ▁Mars , ▁or ▁if ▁it ▁is ▁just ▁a ▁natural ▁landform ."

In [67]:
# 오 이런거네 _ 기준으로 segmenting tokenizing 됐을 때는 segment가 안되서 어색했는데 이제 확실히 낫다 _ 기준으로 나뉘까

['▁H i , ',
 "▁i ' m ",
 '▁Is a ac , ',
 "▁i ' m ",
 '▁going ',
 '▁to ',
 '▁be ',
 '▁writing ',
 '▁about ',
 '▁how ',
 '▁this ',
 '▁face ',
 '▁on ',
 '▁Mars ',
 '▁is ',
 '▁a ',
 '▁natural ',
 '▁landform ',
 '▁or ',
 '▁if ',
 '▁there ',
 '▁is ',
 '▁life ',
 '▁on ',
 '▁Mars ',
 '▁that ',
 '▁made ',
 '▁it . ',
 '▁The ',
 '▁story ',
 '▁is ',
 '▁about ',
 '▁how ',
 '▁NASA ',
 '▁took ',
 '▁a ',
 '▁picture ',
 '▁of ',
 '▁Mars ',
 '▁and ',
 '▁a ',
 '▁face ',
 '▁was ',
 '▁seen ',
 '▁on ',
 '▁the ',
 '▁planet . ',
 '▁NASA ',
 "▁doesn ' t ",
 '▁know ',
 '▁if ',
 '▁the ',
 '▁landform ',
 '▁was ',
 '▁created ',
 '▁by ',
 '▁life ',
 '▁on ',
 '▁Mars , ',
 '▁or ',
 '▁if ',
 '▁it ',
 '▁is ',
 '▁just ',
 '▁a ',
 '▁natural ',
 '▁landform .']

In [75]:
seg_tokens

['▁H i , ',
 "▁i ' m ",
 '▁Is a ac , ',
 "▁i ' m ",
 '▁going ',
 '▁to ',
 '▁be ',
 '▁writing ',
 '▁about ',
 '▁how ',
 '▁this ',
 '▁face ',
 '▁on ',
 '▁Mars ',
 '▁is ',
 '▁a ',
 '▁natural ',
 '▁landform ',
 '▁or ',
 '▁if ',
 '▁there ',
 '▁is ',
 '▁life ',
 '▁on ',
 '▁Mars ',
 '▁that ',
 '▁made ',
 '▁it . ',
 '▁The ',
 '▁story ',
 '▁is ',
 '▁about ',
 '▁how ',
 '▁NASA ',
 '▁took ',
 '▁a ',
 '▁picture ',
 '▁of ',
 '▁Mars ',
 '▁and ',
 '▁a ',
 '▁face ',
 '▁was ',
 '▁seen ',
 '▁on ',
 '▁the ',
 '▁planet . ',
 '▁NASA ',
 "▁doesn ' t ",
 '▁know ',
 '▁if ',
 '▁the ',
 '▁landform ',
 '▁was ',
 '▁created ',
 '▁by ',
 '▁life ',
 '▁on ',
 '▁Mars , ',
 '▁or ',
 '▁if ',
 '▁it ',
 '▁is ',
 '▁just ',
 '▁a ',
 '▁natural ',
 '▁landform .']

In [1]:
token_types

NameError: name 'token_types' is not defined

In [78]:
DataLoader(
        dev_db, batch_size=CFG.batch_size, shuffle=False,
        num_workers=CFG.num_workers, pin_memory=True)

NameError: name 'token_ids' is not defined

In [ ]:
dev_db = cate_dataset.CateDataset(dev_df, img_h5_path, token2id, CFG.seq_len,
                                       CFG.type_vocab_size)

# 여러 개의 워커로 빠르게 배치(미니배치)를 생성하도록 DataLoader로
# CateDataset 인스턴스를 감싸 줌
dev_loader = DataLoader(
    dev_db, batch_size=CFG.batch_size, shuffle=False,
    num_workers=CFG.num_workers, pin_memory=True)

In [ ]:
# dev_loader에서 반복해서 배치 데이터를 받음
# CateDataset의 __getitem__() 함수의 반환 값과 동일한 변수 반환
for step, (token_ids, token_mask, token_types, img_feat, _) in enumerate(dev_loader):

In [94]:
berttokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased',cache_dir='bert_ckpt',)

Downloading:   0%|          | 0.00/972k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.87M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

In [95]:

import numpy as np

In [ ]:
encoded_dict=  tokenizer.encode_plus(
    text = sentence,
    add_special_tokens= True,
    max_length=MAX_LEN,
    pad_to_multiple_of=

)

In [190]:
#token_types

len(token_mask)

4099

In [192]:
token_types

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40]

In [213]:
token_ids = np.array(token_ids, dtype=np.float32)
token_mask = np.array(token_mask,dtype=np.float32)
teokn_types = np.array(token_mask,dtype=np.float32)

In [214]:
text_output = text_encoder(token_ids, token_mask, token_type_ids=token_types)

TypeError: 'int' object is not callable

In [219]:
   # 반환 튜플(시퀀스 아웃풋, 풀드(pooled) 아웃풋) 중 시퀀스 아웃풋만 사용

TypeError: 'int' object is not callable